In [1]:
using System.Collections.Concurrent;
using System.Threading;
using System.Threading.Tasks;

In [2]:
private void Test01()
{
    var cd = new ConcurrentDictionary<int, int>();

    // 將數字 0 到 9999 的存儲在 ConcurrentDictionary 中
    Parallel.For(0, 10000, i =>
    {
        cd[i] = i;
    });

    int sum = 0;

    // 定義一個操作來消耗 ConcurrentDictionary
    Action action = () =>
    {
        int localSum = 0;

        // 從 ConcurrentDictionary 中取出數據
        while (cd.TryRemove(cd.Keys.FirstOrDefault(), out int value))
        {
            localSum += value;
        }

        // 將 localSum 加到 sum 中
        Interlocked.Add(ref sum, localSum);
    };

    // 啟動 4 個並行操作
    Parallel.Invoke(action, action, action, action);
    Console.WriteLine("Sum: {0}", sum);
}

Test01();

Sum: 49995000


In [3]:
private void Test02()
{
    var myDict = new ConcurrentDictionary<string, bool>();

    // 添加一些元素
    myDict.TryAdd("A", false);
    myDict.TryAdd("B", true);
    myDict.TryAdd("C", false);

    myDict["B"] = false;

    // 使用 LINQ 查詢查找是否存在值為 true 的元素
    bool result = myDict.Any(x => x.Value == true);

    Console.WriteLine("result: " + result);
}

Test02();

result: False


In [4]:
private void Test03()
{
    var myDict = new ConcurrentDictionary<string, bool>();

    // 使用 LINQ 查詢查找是否存在值為 true 的元素
    // 因為裡面沒有任何一個元素，因此結果為false
    bool result = myDict.Any(x => x.Value == true);

    // 添加一些元素
    myDict.TryAdd("A", false);
    myDict.TryAdd("B", true);
    myDict.TryAdd("C", false);

    // 第二次的 myDict.TryAdd("A", true) 無效是因為 TryAdd 方法嘗試將指定的索引鍵和值加入至 ConcurrentDictionary<TKey,TValue>
    // 如果索引鍵已經存在，這個方法會傳回 false
    // 由於前面已經成功添加了索引鍵 "A"，所以第二次嘗試添加相同的索引鍵時會失敗。
    myDict.TryAdd("A", true); // 無效

    myDict["B"] = false;

    // 如果字典中不存在該索引鍵，則會自動添加一個新的索引鍵/值對
    // 這個操作不是使用 TryAdd 方法來完成的，而是通過索引器來實現的
    // 根據 MSDN 文檔，ConcurrentDictionary<TKey, TValue> 的所有公共和受保護成員都是線程安全的，可以從多個線程同時使用
    // 這意味著可以在多個線程中安全地使用索引器來訪問和修改字典中的元素，而不需要額外的同步。
    myDict["D"] = true;

    // 使用 LINQ 查詢查找是否存在值為 true 的元素
    result = myDict.Any(x => x.Value == true);

    Console.WriteLine("result: " + result);
}

Test03();

result: True
